# Trading Gas: Bet on Temperature demand

你这个设定非常 G&K：alpha 只在 Temperature demand，其余 6 个是你必须“付房租”的系统性风险。建议的主动组合管理目标就是：

只在你有 edge 的维度上拿风险，其它维度尽量中性；用结构化风险模型把“共同波动”从组合里剥离出来，再用均值-方差（或 tracking-error）优化把 alpha 转成仓位。 ￼

下面给你一套可落地、按 G&K 逻辑闭环的建模与组合框架（我会同时说明每一步在书里对应的思想）。

## 把“世界”写成 7 因子风险模型（你说的那套）

你先把 NG（期货曲线上你可交易的合约/跨期）回报写成结构化模型：
$$
r_t = X_t b_t + u_t
$$

- $X_t$：你定义的 7 个风险因子暴露（exposures / loadings），必须是 a priori（期初已知/可测/可预测的量） ￼
- $b_t$：7 因子收益（真实世界里它们会波动）
- $u_t$：特异项（你没有解释掉的残差）

并且用标准结构化协方差：
$$
V = X F X^\top + A
$$
（$F$：因子协方差；$A$：特异方差/对角） ￼

关键点（G&K 的“风格”）：
风险因子不是为了预测收益，是为了更稳健地预测协方差矩阵并分解风险来源。 ￼

## 把你的 alpha 明确写成：只来自“温度需求的预测误差”

你说 alpha 在 Temperature demand，那你要把它定义成：
$$
\alpha_t = \underbrace{\mathbb{E}[b_{temp,t} \mid \mathcal{I}_t] - \mathbb{E}[b_{temp,t} \mid \mathcal{M}_t]}_{\text{你 vs 市场 的温度因子观点差}}
$$
- $\mathcal{I}_t$：你的信息集（更好的天气模型/更好的需求映射）
- $\mathcal{M}_t$：市场隐含信息（最简化你可以用“当前价格 + 共识天气”做代理）
- 你交易的不是“温度本身”，而是 温度因子收益 $b_{temp}$ 的差异

为了落地，你可以用 HDD/CDD 作为“温度需求”的基础表征（定义在 65°F 基准上）。 

## 组合目标：温度因子有意暴露，其余 6 因子“结构性中性”

这是 G&K 在第三章里强调的：风险模型让你知道边际风险贡献、让你只在有把握处下注。 ￼

给你一个非常实操的约束模板（你可以按 desk 风格放松/收紧）：
- 目标暴露：
- 允许 $x^{port}_{temp}$ 按你的 alpha 强弱动态调整
- 中性约束：
	- $x^{port}_{storage} \approx 0$
	- $x^{port}_{prod} \approx 0$
	- $x^{port}_{lng} \approx 0$
	- $x^{port}_{pipeline/basis} \approx 0$
	- $x^{port}_{powerburn/fuelswitch} \approx 0$
	- $x^{port}_{curve-shape} \approx 0$（或只允许很小的 slope/seasonality 偏离）

直觉：你要做的是 “温度因子主动” + “其余因子被动对冲”。
这比“看多 NG outright”稳定得多，因为你不想把收益归因于 LNG outage、库存 surprise 这种你没有 edge 的东西。

## 交易载体怎么选：用“曲线”制造 breadth，而不是频繁赌方向

你只有一个 alpha 源（温度需求），想把 IR 做上去，按基本定律只能靠：
- 提升 IC（你天气→需求→价格映射更准）
- 或提升 breadth（更多独立下注）

在 NG 上提升 breadth 的最佳方式通常不是加频率，而是加“结构维度”：
- 跨期（calendar spread）：把暴露集中到某个受温度影响更强的月份/季节
- 跨季（winter strip vs shoulder）：让“温度冲击”更像你下注的唯一来源
- 分桶下注：把“冬季 HDD 风险”“夏季 CDD 风险”“肩季小波动”当作不同 bet（基本定律的 additivity 也支持你把不同来源的 IR 平方相加）

## 用“基准 + tracking error”思路，而不是裸 mean-variance

如果你的 mandate 是绝对收益，你可以把“基准”设成 0；
如果你想更 G&K（以及更像机构化）：
- 设一个 风险基准：例如 “平坦曲线的 carry/roll + 0 因子暴露” 的伪基准
- 你管理的是 active return vs benchmark，风险用 active risk / tracking error 表达 ￼

然后做一个带成本的优化：
$$
\max_w \; w^\top \alpha - \lambda \, w^\top V w - \text{TC}(w-w_{prev})
$$
书里也明确提醒：约束和实现摩擦会显著降低你理论 IR（等价于降低 transfer coefficient）。 ￼

## 风险预算：把“你愿意承担的 active risk”绑定到 IC 与 breadth

G&K 的逻辑是：你愿意承担多少主动风险，应该随 skill（IC） 和 breadth（BR） 增长。 ￼
所以实操上我会建议：
- 先在历史回测里估一个 conservative 的 IC（温度需求 alpha 与后续 PnL 的相关）
- 再定一个你能接受的 active risk budget（例如年化 TE 2% / 4% / 6%）
- 用优化器去“用尽”这个预算，但只在 temp 维度

## 数据与“可观测锚点”（你需要让 7 因子都是可操作的）

至少以下三个锚点是我建议你无论如何都要接入并做成 pipeline 的：
- EIA 储气：周度 working gas in storage（方法学与数据都公开且权威） ￼
- 温度需求定义：HDD/CDD 的标准定义（NWS/EIA 都给了很清晰的口径） ￼
- LNG 出口强度：美国 LNG 出口被 EIA 明确认为会抬升 Henry Hub 价格（长期情景分析，但“方向性因子”成立） ￼

同时别忘了合约的基本规格（你做风险预算/仓位换算会用到：10,000 MMBtu/手）。 ￼

## 总结

你可以把这套系统总结成一句话

“用结构化风险模型把 7 因子的共同波动拆开；只在 temperature demand 上留主动暴露；其余因子暴露被约束/对冲；用 tracking-error 优化器在成本与约束下把温度 alpha 转成最有效的曲线仓位。”